# Chapter 11: Training Deep Neural Network

This chapter introduces some common techniques to train a deep neural network, including selecting initializers, choosing activation functions, batch normalization, using better optimizers, etc. This book only contains basic concept and usage of these techniques. For other techniques in training NN or the priciples of these methods, please refer to other materials.

> This jupyter notebook contains my own solution to the coding exercises of the book. However the code may not be optimized to the best performance. For answers to the questions of the book, please check the markdown file under the same directory.

## Exercise 8: Deep Learning

Requirement: 
1. Use 5 hidden layers with 100 neurons, He initializer and ELU activation function to build a DNN.
2. Try to use Adam optimizer and early stopping to train the DNN on MNIST, but only with digit 0 ~ 4, because we need to train the model to recognize digit 5 ~ 9 using transfer learning in the next exercise. You will need an output softmax layer with 5 neurons, and make sure to keep saving the checkpoint for later usage.
3. Use cross validation to fine tune hyperparameters. Check your precision score.
4. Try batch normalization.
5. Does the model overfit the train set? Try adding Dropout for each layer.

In [5]:
import tensorflow as tf

input_tensor = tf.placeholder(tf.float32, shape=(None, 28, 28), name="input")
label = tf.placeholder(tf.float32, shape=(None, 1), name="label")

with tf.name_scope("fully_connected_network"):
    hidden_1 = tf.layers.Dense(100, activation=tf.nn.elu(float32), kernel_initializer=tf.initializers.he_normal(),name="hidden_1")(input_tensor)
    hidden_2 = tf.layers.Dense(100, activation=tf.nn.elu(float32), kernel_initializer=tf.initializers.he_normal(),name="hidden_2")(hidden_1)
    hidden_3 = tf.layers.Dense(100, activation=tf.nn.elu(float32), kernel_initializer=tf.initializers.he_normal(),name="hidden_3")(hidden_2)
    hidden_4 = tf.layers.Dense(100, activation=tf.nn.elu(float32), kernel_initializer=tf.initializers.he_normal(),name="hidden_4")(hidden_3)
    hidden_5 = tf.layers.Dense(100, activation=tf.nn.elu(float32), kernel_initializer=tf.initializers.he_normal(),name="hidden_5")(hidden_4)

# output = tf.nn.softmax_cross_entropy_with_logits
# mse = tf.reduce_mean(tf.square())
# adam_optimizer = tf.train.AdamOptimizer()
# train_op = adam_optimizer.minimize(mse)

init = tf.global_variables_initializer()

# with tf.name_scope("execution"):
    

NameError: name 'float32' is not defined

## Exercise 9: Transfer Learning

Requirement:
1. 

## Exercise 10: Pretrain on Assistant Task

Requirement: